There are 20 trials: X1, X2, ..., X20
keep the depth as 9

Each trial has sequence of points: X1 = {x11, x12, x13, ..., x15}

Algorithm:
    1. upload X1 with given depth for 9 different objects
    2. optimize hyperparameters for all of the models
    3. predict depth+1, record probability, get label
    4. re-upload X1 with new, remove anything far before depth+1

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import pairwise_kernels
from scipy.optimize import fmin_l_bfgs_b
from scipy.linalg import cholesky, cho_solve, solve_triangular
import pickle
import pandas as pd
from scipy.spatial.distance import pdist, squareform, cdist

### Upload data

In [2]:
data = pickle.load(open('../dataset.pkl', 'rb'))
df = pd.read_csv('../df.csv')

In [3]:
df.head()

,thumb_1,thumb_2,thumb_3,thumb_4,thumb_5,thumb_6,thumb_7,thumb_8,thumb_9,thumb_10,...,little_5,little_6,little_7,little_8,little_9,little_10,little_11,little_12,obj_name,trial
0,0.05098,0.050980,0.054902,0.047059,0.047059,0.047059,0.050980,0.047059,0.047059,0.050980,...,0.027451,0.031373,0.027451,0.031373,0.035294,0.031373,0.027451,0.039216,blue_bear,1
1,0.05098,0.050980,0.054902,0.047059,0.047059,0.050980,0.050980,0.050980,0.047059,0.047059,...,0.031373,0.027451,0.027451,0.027451,0.039216,0.027451,0.031373,0.043137,blue_bear,1
2,0.05098,0.054902,0.054902,0.050980,0.050980,0.047059,0.047059,0.050980,0.047059,0.050980,...,0.027451,0.031373,0.027451,0.027451,0.031373,0.035294,0.027451,0.039216,blue_bear,1
3,0.05098,0.054902,0.054902,0.047059,0.050980,0.050980,0.050980,0.050980,0.047059,0.050980,...,0.035294,0.031373,0.027451,0.031373,0.031373,0.035294,0.027451,0.043137,blue_bear,1
4,0.05098,0.054902,0.054902,0.050980,0.050980,0.050980,0.047059,0.050980,0.047059,0.047059,...,0.035294,0.031373,0.027451,0.027451,0.035294,0.027451,0.031373,0.039216,blue_bear,1


60

In [4]:
obj_names = list(df['obj_name'].unique())

In [5]:
obj_to_id = {
    'blue_bear':1,
    'med_coke':2,
    'book':3,
    'empty_coke':4,
    'lotion':5,
    'empty_vitamin_water':6,
    'med_vitamin_water':7,
    'full_vitamin_water':8,
    'monkey_toy':9
}

In [6]:
obj_names

['blue_bear',
 'med_coke',
 'book',
 'empty_coke',
 'lotion',
 'empty_vitamin_water',
 'med_vitamin_water',
 'full_vitamin_water',
 'monkey_toy']

### Kernels

In [7]:
def squared_exp_kernel(X, theta_):
    pairwise_dists = squareform(pdist(X, 'euclidean'))**2
    K = np.exp(-pairwise_dists/ (2 * theta_[0] ** 2))
    K_gradient = np.multiply(pairwise_dists/(theta_[0]**3), K)
    return K, np.expand_dims(K_gradient, axis=2)

def squared_exp_kernel_func(a, b, theta_):
    # theta includes number of parameters
    squared_dist = np.linalg.norm(a-b)**2
    k = np.exp(-squared_dist/(2*theta_[0]**2))
    return k

In [8]:
#from sklearn.gaussian_process.kernels import NormalizedKernelMixin, StationaryKernelMixin, Kernel, Hyperparameter, _check_length_scale

In [12]:
from kernels2 import RBF as RBF2

### Learning

In [13]:
DEPTH = 10

In [17]:
from sklearn.gaussian_process import GaussianProcessRegressor, GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF, ConstantKernel

# Instantiate a Gaussian Process model
kernel = RBF2(np.ones(df.shape[1]-2), (1e-2, 1e2))
gp = GaussianProcessClassifier(kernel=kernel, n_restarts_optimizer=2)

In [18]:

# initilize the first training set
trial = 1
X = np.zeros([1,60])
y = np.ones([1,1])
for obj in obj_names:
    X_temp = data[obj][trial][0:DEPTH]
    y_temp = np.ones(X_temp.shape[0])*obj_to_id[obj]
    y_temp = y_temp.reshape(-1,1)
    X = np.vstack([X, X_temp])
    y = np.vstack([y, y_temp])
    
X = X[1:]
y = y[1:]
gp.fit(X, y)

/home/tasbolat/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianProcessClassifier(copy_X_train=True,
             kernel=RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
             max_iter_predict=100, multi_class='one_vs_rest', n_jobs=1,
             n_restarts_optimizer=2, optimizer='fmin_l_bfgs_b',
             random_state=None, warm_start=False)

In [19]:
obj = 'med_coke'
for i in range(DEPTH, 40):
    X_temp = data[obj][trial][i:i+1]
    if X_temp.shape[0] == 0:
        break
    #y_temp = np.array([1])*obj_to_id[obj]
    #y_temp = y_temp.reshape(-1,1)
    
    # do prediction
    #y_pred, y_sigma = my_gp.predict(theta_= models[obj], x_star=X_temp)
    #print(y_pred, y_sigma)
    
    y_pred = gp.predict(X_temp)
    print(y_pred)
    #X = np.vstack([X, X_temp])
    #y = np.vstack([y, y_temp])
    
    #X = X[1:]
    #y = y[1:]
    

[6.]
[1.]


In [20]:
gp.kernel_

CompoundKernel(1.16, 1.16, 1.03, 1.16, 1.12, 1.16, 1.12, 0.817, 0.817, 1.03, 1.03, 0.817, 1.03, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.03, 0, 0.817, 1.03, 0, 0, 1.23, 1.23, 0.817, 1.12, 1.17, 0, 1.12, 0, 0, 0, 0.817, 0, 1.03, 0, 0, 0, 0.817, 0.817, 1.03, 0, 1.03, 1.24, 1.21, 1.03, 1.03, 1.24, 1.23, 1.21, 1.12, 1.16, 1.16, 1.16, 1.03, 1.16, 1.12, 1.16, 1.12, 0.817, 0.817, 1.03, 1.03, 0.817, 1.03, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.03, 0, 0.817, 1.03, 0, 0, 1.23, 1.23, 0.817, 1.12, 1.17, 0, 1.12, 0, 0, 0, 0.817, 0, 1.03, 0, 0, 0, 0.817, 0.817, 1.03, 0, 1.03, 1.24, 1.21, 1.03, 1.03, 1.24, 1.23, 1.21, 1.12, 1.16, 1.16, 1.16, 1.03, 1.16, 1.12, 1.16, 1.12, 0.817, 0.817, 1.03, 1.03, 0.817, 1.03, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.03, 0, 0.817, 1.03, 0, 0, 1.23, 1.23, 0.817, 1.12, 1.17, 0, 1.12, 0, 0, 0, 0.817, 0, 1.03, 0, 0, 0, 0.817, 0.817, 1.03, 0, 1.03, 1.24, 1.21, 1.03, 1.03, 1.24, 1.23, 1.21, 1.12, 1.16, 1.16, 1.16, 1.03, 1.16, 1.12, 1.16, 1.12, 0.817, 0.817, 1.03, 1.03, 0.817, 1.03, 0, 0, 0

In [59]:
gp.kernel_

CompoundKernel(4.61, 4.61, 4.61, 4.61, 4.61, 4.61, 2.89, 4.61, 4.61)